In [1]:
import torch
import torchvision as tv
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [2]:
transform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.5,),(0.5,)),])

In [3]:
train = tv.datasets.MNIST('./data/torch/', download=True, train=True, transform=transform)
val = tv.datasets.MNIST('./data/torch/', download=True, train=False, transform=transform)
trainLoader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
valLoader = torch.utils.data.DataLoader(val, batch_size=128, shuffle=True)

In [4]:
model = tv.models.resnet50()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(in_features=2048, out_features=10, bias=True)
if(torch.cuda.is_available()):
    print('Using GPU')
    model = model.cuda()

Using GPU


In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
print('Training....')
total = 0
correct = 0
start = time.time()
epochs = 20

for epoch in range(epochs):
    
    for i, data in enumerate(trainLoader, 1):
        images, labels = data

        if(torch.cuda.is_available()):
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()    
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        if(i%100 == 0):
            print('Epoch: {} Batch: {} loss: {}'.format(epoch, i, loss.item()))

        loss.backward()
        optimizer.step()

print('Training Completed in: {} secs'.format(time.time()-start))
print('Training accuracy: {} %'.format((correct/total)*100))

Training....
Epoch: 0 Batch: 100 loss: 0.8476041555404663
Epoch: 0 Batch: 200 loss: 0.4154044985771179
Epoch: 0 Batch: 300 loss: 0.3029526472091675
Epoch: 0 Batch: 400 loss: 0.16123035550117493
Epoch: 1 Batch: 100 loss: 0.043583132326602936
Epoch: 1 Batch: 200 loss: 0.17669670283794403
Epoch: 1 Batch: 300 loss: 0.0555841438472271
Epoch: 1 Batch: 400 loss: 0.10364382714033127
Epoch: 2 Batch: 100 loss: 0.1637398898601532
Epoch: 2 Batch: 200 loss: 0.1070326417684555
Epoch: 2 Batch: 300 loss: 0.09405642002820969
Epoch: 2 Batch: 400 loss: 0.0599663071334362
Epoch: 3 Batch: 100 loss: 0.01920899748802185
Epoch: 3 Batch: 200 loss: 0.012079800479114056
Epoch: 3 Batch: 300 loss: 0.06716079264879227
Epoch: 3 Batch: 400 loss: 0.04215865954756737
Epoch: 4 Batch: 100 loss: 0.022891538217663765
Epoch: 4 Batch: 200 loss: 0.059664350003004074
Epoch: 4 Batch: 300 loss: 0.02592439576983452
Epoch: 4 Batch: 400 loss: 0.008183004334568977
Epoch: 5 Batch: 100 loss: 0.0024268836714327335
Epoch: 5 Batch: 200 l

In [7]:
print('Predicting....')
start = time.time()

predictions = torch.LongTensor()
allLabels = torch.LongTensor()
for i, data in enumerate(valLoader, 1):
    images, labels = data

    if(torch.cuda.is_available()):
        images = images.cuda()
        labels = labels.cuda()

    if(i%100 == 0):
        print('Batch {} done'.format(i))
    outputs = model(images)
    
    pred = outputs.cpu().data.max(1, keepdim=True)[1]
    predictions = torch.cat((predictions, pred), dim=0)
    allLabels = torch.cat((allLabels, labels), dim=0)
    
print('Completed in {} secs'.format(time.time() - start))

Predicting....
Completed in 2.4802470207214355 secs


In [8]:
classification_report(y_pred=predictions, y_true=allLabels)
cm = confusion_matrix(allLabels, predictions)
ConfusionMatrixDisplay(cm).plot(cmap='Blues', xticks_rotation='vertical', values_format='d')

tensor([[5],
        [3],
        [0],
        ...,
        [8],
        [8],
        [3]])

In [9]:
torch.save(model, './models/resnet50-mnist.pt')